In [1]:
import os
from modelarts.session import Session
sess = Session()
bucket_path="grp-homework/GRP.tar.gz"
sess.download_data(bucket_path=bucket_path, path="./GRP.tar.gz")
# 使用tar命令解压资源包
os.system("tar -xf ./GRP.tar.gz")
# 清理压缩包
os.system("rm ./GRP.tar.gz")

Successfully download file grp-homework/GRP.tar.gz from OBS to local ./GRP.tar.gz


0

In [2]:
!pip install xlwt
import xlwt
import glob
import datetime

You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# 创建一个workbook 设置编码
workbook = xlwt.Workbook(encoding = 'utf-8')
# 创建一个worksheet
worksheet = workbook.add_sheet('GRP_res')

# 写入excel
# 参数对应 行, 列, 值
worksheet.write_merge(0, 1, 0, 0, '名')
worksheet.write_merge(0, 1, 1, 1, 'Ⅰ类')
worksheet.write_merge(0, 1, 2, 2, 'Ⅱ类')
worksheet.write_merge(0, 0, 3, 6, 'dlib')
worksheet.write(1,3, label = '识别数')
worksheet.write(1,4, label = '正确数')
worksheet.write(1,5, label = '漏检数')
worksheet.write(1,6, label = '错检数')
worksheet.write_merge(0, 0, 7, 10, 'MTCNN')
worksheet.write(1,7, label = '识别数')
worksheet.write(1,8, label = '正确数')
worksheet.write(1,9, label = '漏检数')
worksheet.write(1,10, label = '错检数')
worksheet.write_merge(0, 0, 11, 14, 'OpenCv')
worksheet.write(1,11, label = '识别数')
worksheet.write(1,12, label = '正确数')
worksheet.write(1,13, label = '漏检数')
worksheet.write(1,14, label = '错检数')
a=1
b=1
c=0
ps=1
while ((a!=7)or(b!=1)or(c!=0)):
    ps+=1
    c+=1
    worksheet.write(ps,0, label = a*100+b*10+c)
    worksheet.write(ps,1, label = a)
    worksheet.write(ps,2, label = b)
    if((b==1)or(b==2)):
        if(c==5):
            c=0
            b+=1
    elif(b==3):
        if(c==3):
            c=0
            b+=1
    elif(b==4):
        if(c==2):
            c=0
            b+=1
    if(b==5):
        b=1
        a+=1

In [4]:
#dlib preparation

import os
from modelarts.session import Session
sess = Session()

if sess.region_name == 'cn-north-1':
    bucket_path="modelarts-labs/notebook/DL_face_detector/face_detector.tar.gz"
elif sess.region_name == 'cn-north-4':
    bucket_path="modelarts-labs-bj4/notebook/DL_face_detector/face_detector.tar.gz"
else:
    print("请更换地区到北京一或北京四")

if not os.path.exists('./models/detector.dat'):    
    sess.download_data(bucket_path=bucket_path, path="./face_detector.tar.gz")

if os.path.exists('./face_detector.tar.gz'):
    # 使用tar命令解压资源包
    os.system("tar -xf ./face_detector.tar.gz")
    # 清理压缩包
    os.system("rm ./face_detector.tar.gz")

!pip install dlib==19.17.0
from PIL import Image
import dlib
import cv2

cnn_face_detector = dlib.cnn_face_detection_model_v1("./models/detector.dat")

Successfully download file modelarts-labs-bj4/notebook/DL_face_detector/face_detector.tar.gz from OBS to local ./face_detector.tar.gz
You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
# dlib action
startTime = datetime.datetime.now()
for index, filename in enumerate(glob.glob('GRP/my_data/' + '*.jpg')):
    image = dlib.load_rgb_image(filename)
    dets = cnn_face_detector(image, 1)
    rects = dlib.rectangles()
    rects.extend([d.rect for d in dets])
    res_img = image
    for x in rects:
        res_img = cv2.rectangle(image, (x.left(), x.top()), (x.right(), x.bottom()), (0,255,0),2)
    pic=Image.fromarray(res_img)
    cstr=filename[12:15]
    pic.save('GRP/dlib_res/'+cstr+".png")
    DNA=(int(cstr[0])-1)*15+int(cstr[2])
    if int(cstr[1])==2:
        DNA +=5
    elif int(cstr[1])==3:
        DNA +=10
    elif int(cstr[1])==4:
        DNA +=13
    worksheet.write(DNA+1,3, label = len(rects))
endTime = datetime.datetime.now()
print(endTime-startTime)

0:00:13.646997


In [6]:
#MTCNN preparation
!pip install mtcnn==0.0.8
import numpy as np
import cv2
import tensorflow as tf
import random
from PIL import Image
from mtcnn.mtcnn import MTCNN as mtcnn
detector=mtcnn()

You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
#MTCNN action
startTime = datetime.datetime.now()
for index, filename in enumerate(glob.glob('GRP/my_data/' + '*.jpg')):
    image_path = filename
    img = Image.open(image_path)
    img = np.array(img)
    detected = detector.detect_faces(img)
    res_img = img##
    for x in detected:
        box = x["box"]
        res_img = cv2.rectangle(img, (box[0],box[1]),(box[0]+box[2],box[1]+box[3]),(0,255,0),2)
    pic = Image.fromarray(res_img)
    cstr=filename[12:15]
    pic.save('GRP/MTCNN_res/'+cstr+".png")
    DNA=(int(cstr[0])-1)*15+int(cstr[2])
    if int(cstr[1])==2:
        DNA +=5
    elif int(cstr[1])==3:
        DNA +=10
    elif int(cstr[1])==4:
        DNA +=13
    worksheet.write(DNA+1,7, label = len(detected))
endTime = datetime.datetime.now()
print(endTime-startTime)

0:00:15.875764


In [8]:
#OpenCv preparation
import cv2 as cv
from PIL import Image
path="/home/ma-user/anaconda3/envs/TensorFlow-1.13.1/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_alt.xml"
face_cascade = cv.CascadeClassifier(path)

In [9]:
#OpenCv action
startTime = datetime.datetime.now()
for index, filename in enumerate(glob.glob('GRP/my_data/' + '*.jpg')):
    #修改为自己的图片文件名
    image_path = filename
    img = cv.imread(image_path)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    for(x,y,w,h) in faces:
        cv.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    img = cv.cvtColor(img,cv.COLOR_BGR2RGB)
    pic=Image.fromarray(img)
    cstr=filename[12:15]
    pic.save('GRP/OpenCv_res/'+cstr+".png")
    DNA=(int(cstr[0])-1)*15+int(cstr[2])
    if int(cstr[1])==2:
        DNA +=5
    elif int(cstr[1])==3:
        DNA +=10
    elif int(cstr[1])==4:
        DNA +=13
    worksheet.write(DNA+1,11, label = len(faces))
endTime = datetime.datetime.now()
print(endTime-startTime)

0:00:12.442085


In [10]:
workbook.save('GRP/GRP_res.xls')